In [0]:
import urllib.request
urllib.request.urlretrieve("https://www.gutenberg.org/files/1661/1661-0.txt" ,"/tmp/sherlock_holmes.txt")
dbutils.fs.mv("file:/tmp/sherlock_holmes.txt","dbfs:/data/sherlock_holmes.txt")

Out[8]: True

In [0]:
from pyspark import SparkContext , SparkConf
conf = SparkConf().setAppName("word count")
sc = SparkContext.getOrCreate(conf = conf)
bookrdd = sc.textFile("/data/sherlock_holmes.txt")

In [0]:
bookrdd = bookrdd.flatMap(lambda x : x.lower().strip().split(" "))
bookrdd = bookrdd.filter(lambda x : len(x)!=0)

In [0]:
bookrdd.collect()

Out[40]: ['the',
 'project',
 'gutenberg',
 'ebook',
 'of',
 'the',
 'adventures',
 'of',
 'sherlock',
 'holmes,',
 'by',
 'arthur',
 'conan',
 'doyle',
 'this',
 'ebook',
 'is',
 'for',
 'the',
 'use',
 'of',
 'anyone',
 'anywhere',
 'in',
 'the',
 'united',
 'states',
 'and',
 'most',
 'other',
 'parts',
 'of',
 'the',
 'world',
 'at',
 'no',
 'cost',
 'and',
 'with',
 'almost',
 'no',
 'restrictions',
 'whatsoever.',
 'you',
 'may',
 'copy',
 'it,',
 'give',
 'it',
 'away',
 'or',
 're-use',
 'it',
 'under',
 'the',
 'terms',
 'of',
 'the',
 'project',
 'gutenberg',
 'license',
 'included',
 'with',
 'this',
 'ebook',
 'or',
 'online',
 'at',
 'www.gutenberg.org.',
 'if',
 'you',
 'are',
 'not',
 'located',
 'in',
 'the',
 'united',
 'states,',
 'you',
 'will',
 'have',
 'to',
 'check',
 'the',
 'laws',
 'of',
 'the',
 'country',
 'where',
 'you',
 'are',
 'located',
 'before',
 'using',
 'this',
 'ebook.',
 'title:',
 'the',
 'adventures',
 'of',
 'sherlock',
 'holmes',
 'author:',